# Imports

In [31]:
import pandas as pd
import os
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, f1_score

import warnings
warnings.filterwarnings("ignore")

# Feature View and Training Dataset Retrieval

## Connecting to Hopsworks Feature Store

In [2]:
!pip install hopsworks

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import hopsworks

project = hopsworks.login() 

fs = project.get_feature_store() 

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated

Paste it here: ··········
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5497
Connected. Call `.close()` to terminate connection gracefully.


## Get Data

In [4]:
feature_view = fs.get_feature_view(
    name = 'miami_data_air_quality_fv',
    version = 1
)

In [5]:
train_data = feature_view.get_training_data(1)[0]

train_data.head()

,city_x,date,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,cloudcover,visibility,solarradiation,solarenergy,uvindex,conditions,city_y,aqi,o3_mean,pm25_mean
0,Miami,1669507200000,30.7,24.1,26.8,33.7,24.1,28.3,22.2,76.6,...,50.1,16.0,131.5,11.3,6.0,Partially cloudy,Miami,29,0.020,2.9
1,Miami,1671494400000,26.5,20.6,22.8,26.5,20.6,22.8,18.0,75.0,...,71.0,15.7,96.5,8.3,5.0,"Rain, Partially cloudy",Miami,39,0.039,3.3
2,Miami,1670371200000,28.4,21.0,24.0,29.5,21.0,24.3,18.7,72.9,...,39.9,16.0,103.0,8.8,6.0,"Rain, Partially cloudy",Miami,37,0.037,3.3
3,Miami,1669420800000,30.2,23.4,26.4,33.5,23.4,27.6,22.3,79.1,...,36.2,16.0,126.7,10.9,6.0,Partially cloudy,Miami,35,0.035,3.5
4,Miami,1670544000000,27.4,19.8,23.5,28.9,19.8,23.6,19.2,77.4,...,46.9,16.0,123.3,10.6,6.0,Partially cloudy,Miami,35,0.035,3.5


In [112]:
fs = project.get_feature_store() 
feature_view = fs.get_feature_view(
    name = 'miami_data_air_quality_fv',
    version = 1
)
train_data = feature_view.get_training_data(1)[0]
train_data = train_data.drop(labels = 'city_y',axis =1)
train_data = train_data.rename(columns = {'city_x':'city'})
train_data = train_data.sort_values(by="date", ascending=True).reset_index(drop=True)
train_data["aqi_next_day"] = train_data.groupby('city')['aqi'].shift(1)

train_data.head(5)
X = train_data.drop(columns=["date"]).fillna(0)
y = X.pop("aqi_next_day")
X = X.drop(columns =['city','conditions']).fillna(0)

today_data = X[1:2]
y = model.predict(today_data)
y

Connected. Call `.close()` to terminate connection gracefully.


array([44.])

In [111]:
y = model.predict(today_data)
y

array([ 0., 44., 28., 31.])

In [105]:
today_data = X[1:2]

# Training

## Modeling

In [97]:
# train_data = train_data.drop(labels = 'city_y',axis =1)
train_data = train_data.rename(columns = {'city_x':'city'})
train_data = train_data.sort_values(by="date", ascending=True).reset_index(drop=True)
train_data["aqi_next_day"] = train_data.groupby('city')['aqi'].shift(1)

train_data.head(5)

,city,date,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,visibility,solarradiation,solarenergy,uvindex,conditions,city_y,aqi,o3_mean,pm25_mean,aqi_next_day
0,Miami,1664668800000,29.8,23.9,26.7,30.9,23.9,27.1,20.0,68.7,...,15.7,159.7,13.8,8.0,"Rain, Partially cloudy",Miami,44,0.044,2.7,NaN
1,Miami,1664755200000,31.0,22.5,26.8,31.1,22.5,27.2,18.3,60.7,...,16.0,158.3,13.6,7.0,Partially cloudy,Miami,97,0.097,3.0,44.0
2,Miami,1664841600000,30.8,23.4,26.6,31.1,23.4,26.9,19.0,64.0,...,16.0,141.4,12.1,9.0,Partially cloudy,Miami,49,0.049,3.0,97.0
3,Miami,1664928000000,29.7,22.6,25.9,30.6,22.6,26.5,19.7,69.8,...,16.0,184.1,15.9,8.0,"Rain, Partially cloudy",Miami,42,0.042,2.1,49.0
4,Miami,1665014400000,28.7,23.5,26.2,30.6,23.5,26.9,19.6,67.6,...,16.0,194.4,16.8,8.0,Partially cloudy,Miami,37,0.037,2.0,42.0


In [83]:
X = train_data.drop(columns=["date"]).fillna(0)
y = X.pop("aqi_next_day")
X = X.drop(columns =['city','conditions']).fillna(0)

In [84]:
X

,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,...,winddir,pressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,aqi,o3_mean,pm25_mean
0,29.8,23.9,26.7,30.9,23.9,27.1,20.0,68.7,0.504,100.0,...,345.4,1012.3,86.7,15.7,159.7,13.8,8.0,44,0.044,2.7
1,31.0,22.5,26.8,31.1,22.5,27.2,18.3,60.7,0.000,0.0,...,339.7,1013.2,60.6,16.0,158.3,13.6,7.0,97,0.097,3.0
2,30.8,23.4,26.6,31.1,23.4,26.9,19.0,64.0,0.000,0.0,...,8.2,1013.6,58.6,16.0,141.4,12.1,9.0,49,0.049,3.0
3,29.7,22.6,25.9,30.6,22.6,26.5,19.7,69.8,0.128,100.0,...,30.5,1015.1,49.4,16.0,184.1,15.9,8.0,42,0.042,2.1
4,28.7,23.5,26.2,30.6,23.5,26.9,19.6,67.6,0.000,0.0,...,42.3,1017.0,40.6,16.0,194.4,16.8,8.0,37,0.037,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,26.5,18.2,21.9,26.5,18.2,21.9,15.6,70.0,0.000,0.0,...,321.8,1019.4,21.2,16.0,125.5,10.8,6.0,46,0.046,4.5
99,26.3,17.4,21.0,26.3,17.4,21.0,13.0,63.2,0.000,0.0,...,302.7,1018.9,15.9,16.0,137.7,11.9,6.0,43,0.043,3.2
100,25.9,12.9,19.2,25.9,12.9,19.2,11.4,63.0,0.000,0.0,...,8.5,1020.7,9.5,16.0,141.4,12.2,6.0,54,0.054,4.8
101,26.3,15.7,20.9,26.3,15.7,20.9,14.5,68.4,0.000,0.0,...,140.9,1020.5,28.6,16.0,142.3,12.4,7.0,45,0.045,4.5


In [85]:
x_train, x_test, y_train, y_test = train_test_split(X,y)

In [86]:

regressor = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    max_depth=3
)
regressor.fit(X, y)

[15:47:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor()

In [87]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=2)

In [88]:
model.fit(x_train, y_train.values.ravel())

KNeighborsClassifier(n_neighbors=2)

In [89]:
y_pred = model.predict(x_test)
print(y_pred)


[36. 25. 37. 25. 36. 32. 31. 27. 33. 31. 32. 36. 25. 36. 31. 35. 25. 31.
 43. 35. 31. 36. 31. 40. 31. 36.]


In [106]:
y = model.predict(today_data)
y

array([44.])

In [90]:
x_test

,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,...,winddir,pressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,aqi,o3_mean,pm25_mean
62,27.6,19.9,23.5,28.0,19.9,23.6,16.2,64.3,0.000,0.0,...,30.1,1020.5,31.4,16.0,126.2,11.0,6.0,38,0.038,3.3
44,29.7,22.5,26.0,32.3,22.5,27.0,21.2,75.4,0.000,0.0,...,114.3,1018.6,31.8,16.0,152.8,13.2,7.0,28,0.028,2.6
34,29.7,23.3,26.2,31.7,23.3,26.9,20.3,70.8,0.778,100.0,...,77.4,1013.8,44.6,15.9,136.0,11.6,7.0,42,0.042,2.5
78,26.5,20.6,22.8,26.5,20.6,22.8,18.0,75.0,7.467,100.0,...,55.7,1020.5,71.0,15.7,96.5,8.3,5.0,39,0.039,3.3
26,30.0,21.8,26.1,32.4,21.8,26.9,19.7,68.3,0.000,0.0,...,54.6,1017.4,27.2,16.0,138.4,11.9,7.0,34,0.034,2.6
67,27.4,19.8,23.5,28.9,19.8,23.6,19.2,77.4,0.000,0.0,...,359.5,1019.1,46.9,16.0,123.3,10.6,6.0,35,0.035,3.5
29,30.7,24.2,27.4,35.2,24.2,29.5,23.1,78.2,0.449,100.0,...,104.3,1014.4,43.4,16.0,118.0,10.1,5.0,31,0.025,3.1
92,27.5,22.9,24.9,29.5,22.9,25.3,20.9,78.7,0.000,0.0,...,118.6,1020.6,48.6,16.0,136.5,11.8,6.0,27,0.027,2.2
102,27.3,15.2,19.9,27.3,15.2,19.9,14.4,71.3,1.046,100.0,...,260.6,1014.5,38.8,15.9,105.2,9.3,6.0,43,0.039,4.3
45,26.5,21.0,23.3,26.5,21.0,23.3,19.4,79.6,0.000,0.0,...,347.5,1019.1,74.6,15.3,90.2,7.8,7.0,42,0.036,4.2


In [91]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
metrics = classification_report(y_test, y_pred, output_dict=True)
results = confusion_matrix(y_test, y_pred)
print(metrics)
print(results)

{'25.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0.0}, '26.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1.0}, '27.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0.0}, '28.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2.0}, '31.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0.0}, '32.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0.0}, '33.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1.0}, '34.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1.0}, '35.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 3.0}, '36.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1.0}, '37.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1.0}, '38.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2.0}, '39.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1.0}, '40.0': {'p

## Validation

Mean Square Error

In [92]:
y_pred = regressor.predict(x_test)

MSE = mean_squared_error(y_test, y_pred)

MSE

4.483332606848657

F1 Score

In [93]:
f1_score(y.astype('int'),[int(pred) for pred in regressor.predict(X)],average='micro')

0.23300970873786409

In [94]:
y.iloc[4:10].values

array([42., 37., 45., 31., 25., 28.])

In [95]:
pred_df = pd.DataFrame({
    'aqi_real': y.iloc[4:10].values,
    'aqi_pred': map(int, regressor.predict(X.iloc[4:10]))
}
)
pred_df

,aqi_real,aqi_pred
0,42.0,41
1,37.0,38
2,45.0,42
3,31.0,30
4,25.0,24
5,28.0,28


# Model Registry

In [73]:
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


In [74]:
model_dir="air_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

In [75]:
import joblib
joblib.dump(model, model_dir + "/air_model3.pkl")

['air_model/air_model3.pkl']

## Model Schema

In [76]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X)
output_schema = Schema(y)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)


In [41]:
# import joblib
# model_dir="weather_model"
# joblib.dump(regressor, 'model.pkl')

In [77]:
air_model = mr.python.create_model(
      name="air_model_3", 
      model_schema=model_schema,
      description="Iris Flower Predictor"
  )
    
air_model.save(model_dir)

  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://c.app.hopsworks.ai:443/p/5497/models/air_model_3/1


Model(name: 'air_model_3', version: 1)